# Notebook para visualizar los datos extraídos desde la API

En este notebook se crearán diversos gráficos para visualizar los datos descargados desde la API y poder realizar una evaluación de los mismos antes de hacer cualquier representación en Streamlit, framework de Python que permite generar aplicaciones de datos interactivas.

Cargamos las librerias a usar para el manejo de datos, como pandas, y para pintar gráficos como Plotly o Seaborn

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

#### Clasificación por pilotos
Representación en un gráfico de líneas, con seaborn, los puntos obtenidos durante una temporada sumando los puntos de carreras y carreras formato sprint 

In [ ]:
# Cargar los datos de resultados en un dataframe
formatted_results = pd.read_csv('../data/race_and_sprint_results_2000-2024.csv')

# Ordenar los registros por temporada, carrera (round) y piloto
formatted_results = formatted_results.sort_values(by=['season', 'round', 'driverId'])

# Realizar la suma acumulativa de los puntos obtenidos por piloto. Para cada carrera se calculará el total te puntos obtenidos por piloto hasta esa carrera
formatted_results['cumulative_points'] = formatted_results.groupby(['season','driverId'])['points'].cumsum()

# Crear el lineplot con seaborn
plt.figure(figsize=(14, 8))
sns.lineplot(
    data=formatted_results[formatted_results.season == 2024], 
    x='circuitId', # Circuitos en el eje X
    y='cumulative_points', # Suma de puntos obtenidos en el eje Y
    hue='driverId',
    marker='o',
    palette='tab20'      
)

# Personalizar el gráfico
plt.title('Puntos Ganados por Piloto en Cada Carrera')
plt.xlabel('Carrera')
plt.ylabel('Puntos Ganados')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Piloto', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.ylim(0,425)
plt.yticks(range(0, 425, 25))  # Intervalo y salto de valores en el eje y

# Mostrar gráfico
plt.show()

#### Clasificación por constructores 
En esta sección, se prueba a crear un gráfico de barras para la visualización de constructores. Para ello, se tiene en consideración los puntos obtenidos por los pilotos que pertenecen a cada constuctor a lo largo de la temporada


Definición de un diccionario para establecer un color por constructor acorde a los colores que les representa en la realidad

In [ ]:
constructor_color_dict = {
    'Alfa Romeo': '#9C2D2C',      
    'AlphaTauri': '#1E1E1E',     
    'Alpine F1 Team': '#0070BB', 
    'Aston Martin': '#006F42',    
    'Caterham': '#006747',       
    'Ferrari': '#DC0000',         
    'Force India': '#5F4B8B',     
    'Haas F1 Team': '#666666',   
    'HRT': '#F2A900',              
    'Lotus F1': '#1E1E1E',        
    'Lotus': '#F4E300',           
    'Manor Marussia': '#E60012',  
    'Marussia': '#A00000',       
    'McLaren': '#FF5700',         
    'Mercedes': '#00D2BE',        
    'Racing Point': '#F9A9D5',    
    'RB F1 Team': '#1E41FF',      
    'Red Bull': '#1E41FF',        
    'Renault': '#FFCD00',        
    'Sauber': '#003A5C',          
    'Toro Rosso': '#1E41FF',      
    'Virgin': '#E10000',          
    'Williams': '#0046A3'         
}

In [ ]:
# Usar mismo dataframe que en el gráfico anterior pero filtrando por la temporada 2024
formatted_results_2024 = formatted_results[formatted_results.season == 2024]
# Crear nuevo dataframe con las columnas necesarias para crear el gráfico
columns_for_constructos = ['season', 'constructorId', 'constructorName', 'constructorNationality', 'points', 'weekendPoints']
filtered_constructors_df = formatted_results_2024[columns_for_constructos].copy()

# Sumar los puntos a partir de agrupar las filas por el constructor y ordenamos el dataframe por los puntos de manera ascendente
grouped_constructors_df = filtered_constructors_df.groupby(['season','constructorId', 'constructorName'])['weekendPoints'].sum().reset_index()
ordered_constructors_df = grouped_constructors_df.sort_values(by='weekendPoints', ascending=True)

fig_constructors = px.bar(
    ordered_constructors_df, 
    x='weekendPoints', # Puntuación
    y='constructorName', # Constructor
    orientation='h', # Barras verticales
    title=f'Clasificación de constructores en la temporada',
    labels={'weekendPoints': 'Points', 'constructorName': 'Constructor'}, # Labels de las barras cuando se hace hover
    color='constructorName',
    color_discrete_map=constructor_color_dict # Usamos el diccioario definido con los colores de los constructores para colorear las barras
)
# Mostrar gráfico
fig_constructors.show()


#### Victorias

A partir del mismo conjunto de datos, se extraen los pilotos que han quedado en primera posición durante la temporada y se realiza su representación en un gráfico circular con plotly.

In [ ]:
# Resultados para la temporada 2024
race_results = formatted_results[formatted_results['season'] == 2024]

# Filas con posición = 1 representa las victorias
wins = race_results[race_results['position'] == 1]
wins.columns

In [ ]:
# Agrupar por las columnas que contienen la información del piloto y utilizar size() para obtener el número de elementos tras la agrupación
grouped_wins = wins.groupby(['driverCode', 'driverName', 'driverSurname']).size().reset_index(name='Wins')
grouped_wins = grouped_wins.sort_values(by='Wins', ascending=False)
# Nueva columna con el nombre completo del piloto
grouped_wins['driverFullName'] = grouped_wins['driverName'] + " " + grouped_wins['driverSurname']
# Crear gráfico de pie
fig = px.pie(
    grouped_wins,
    names='driverFullName', # Nombre de los pilotos
    values='Wins', # Número de victorias
    title="Distribución de Victorias por Piloto",
    hole=0.2 # Tamaño del agujero central del gráfico
)
# Mostrar gráfico de pie
fig.show()

#### Pódiums

En esta sección se pretende realizar otro gráfico de Pie similar al de la sección de Victorias pero ahora mostrando la distribución de las posiciones de pódium para una temporada

In [ ]:
# Dataframe con filas cuya posición es inferior a 4, es decir, posiciones de podium
podiums = race_results[race_results['position'] < 4]

# Agrupar por las columnas que contienen la información del piloto y utilizar size() para obtener el número de elementos tras la agrupación
grouped_podiums = podiums.groupby(['driverCode', 'driverName', 'driverSurname']).size().reset_index(name='Podiums')
grouped_podiums = grouped_podiums.sort_values(by='Podiums', ascending=False)
# Nueva columna con el nombre completo del piloto
grouped_podiums['driverFullName'] = grouped_podiums['driverName'] + " " + grouped_podiums['driverSurname']
# Crear gráfico de pie
fig = px.pie(
    grouped_podiums,
    names='driverFullName', # Nombre de los pilotos  
    values='Podiums',  # Número de pódiums
    title="Distribución de Pódiums por Piloto",
    hole=0.2 # Tamaño del agujero central del gráfico
)
fig.update_layout(
    height=400
)
fig.show()

#### Pitstops

Parte experimental en la que a partir del conjunto de datos del histórico de pitstops se han realizado una serie de representaciones considerando los tiempos.

In [ ]:
# Cargar tiempos de pitstops en un dataframe
pitstops_df = pd.read_csv('../data/cleaned_pitstops.csv')
pitstops_df.head()

In [ ]:
# Función para convertir los tiempos de pitstops a segundos
def duration_to_seconds(duration):
    duration_str = str(duration)
    if ':' in duration_str: 
        minutes, seconds_miliseconds = duration_str.split(':')
        seconds, miliseconds = seconds_miliseconds.split('.')
        return int(minutes) * 60 + int(seconds) + int(miliseconds) / 1000
    else:  
        return float(duration_str)

In [ ]:
# Realizar conversión
pitstops_df['duration'] = pitstops_df['duration'].apply(lambda duration: duration_to_seconds(duration))

Duración de los pitstops entre la temporada 2011 y 2024. Cogemos todos los registros con duración inferior a 50 segundos, ya que todo lo que supere ese valor no son tiempos razonables y pueden deberse a circunstancias anormales.

Gráfico de dispersión de los pitstops. En el eje X la temporada y en el eje Y la duración. Los colores de los puntos dependen del constructor.

In [ ]:
fig = px.scatter(pitstops_df[pitstops_df['duration']<50],
                 x='date',
                 y='duration',
                 color='constructorName',
                 color_discrete_map=constructor_color_dict,
                )
fig.show()

Gráfico de cajas que muestra la distribución de los tiempos de pitstop por constructor para la temporada 2024

In [ ]:
pitstops_2024_df = pitstops_df[pitstops_df['season'] == 2024]

fig = px.box(
    pitstops_2024_df[pitstops_2024_df['duration']<50],
    x='constructorName',
    y='duration',
    color='constructorName',
    color_discrete_map=constructor_color_dict,
)
fig.show()

Gráfico de cajas que muestra la distribución de los tiempos de pitstop por circuito para la temporada 2024

In [ ]:
# Pit stop duration per circuit
fig = px.box(
    pitstops_df[pitstops_df['duration']<50],
    x='raceName',
    y='duration'
)
fig.show()

Histograma que muestra la distribución de los tiempos de pitstops en segundos

In [ ]:
fig = px.histogram(
    pitstops_df[pitstops_df['duration']<50], 
    x="duration"
)
fig.show()